<a href="https://colab.research.google.com/github/sturu1/Mon-AM/blob/master/2020_08_03_%EC%98%A4%ED%9B%84%EC%88%98%EC%97%85_%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%EB%A3%A8%ED%94%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

트레이닝 루프

# Setup

In [52]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [53]:
# batch_size = 64
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# x_train = np.reshape(x_train, (-1, 784))
# x_test = np.reshape(x_train, (-1, 784))
# train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.shuffle(buffer_size = 1024).batch(batch_size)

In [54]:
#모델정의
inputs = keras.Input(shape = (784, ), name = "digits")
x = layers.Dense(64, activation = "relu", name = "dense_1")(inputs)
x = layers.Dense(64, activation = "relu", name = "dense_2")(x)
outputs = layers.Dense(10, name = "prediotions")(x)
model = keras.Model(inputs = inputs, outputs = outputs)

# 최적화 인스턴스
optimizer = keras.optimizers.SGD(learning_rate = 1e-3)

#손실함수 정의
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits = True)

#정확도 측정 인스턴스
train_acc_metric = keras.metrics.SparseCategoricalCrossentropy()
val_acc_metric = keras.metrics.SparseCategoricalCrossentropy()

#훈련데이터셋 준비
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size = 1024).batch(batch_size)

#10000개 데이터셋은 검증데이터셋으로 설정
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

# @tf.function 데코레이션을 붙이면 학습속도가 빨라진다.

 gradient를 구해서 가중치를 업데이트하는 부분이 학습의 핵심이라 볼 수 잇고, 그 부분만 함수로 따로 뺸 다음 @tf.function을 붙여주면 빨라진다.

In [55]:
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    logits = model(x, training = True)
    loss_value = loss_fn(y, logits)
  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  train_acc_metric.update_state(y, logits)
  return loss_value

In [57]:
@tf.function
def test_step(x, y):
  val_logits = model(x, training = False)
  val_acc_metric.update_state(y, val_logits)

In [58]:
import time

epochs = 2
for epoch in range(epochs):
  print("\n%d번째 epoch 시작" % (epoch, ))
  start_time = time.time()
  #
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
    loss_value = train_step(x_batch_train, y_batch_train)
    #
    if step % 200 == 0:
      print("Training loss (for one batch) at step %d 번째 스텝에서 배치별 훈련데이터 손실: %.4f" % (step, float(loss_value)))
      print("현재까지 학습한 데이터 개수: %d" % ((step + 1) * 64))

  #Display metrics at the end each epoch.
  train_acc =train_acc_metric.result()
  print("epoch별 학습데이터 정확도: %.4f" % (float(train_acc), ))

  # Reset training metrics at the end of each epoch
  train_acc_metric.reset_states()
  
  # Run a validation loop at the end of each epoch.
  for x_batch_val, y_batch_val in val_dataset:
    test_step(x_batch_val, y_batch_val)

  val_acc = val_acc_metric.result()
  val_acc_metric.reset_states()
  print("검증데이터 정확도 : %.4f" % (float(val_acc), ))
  print("걸린 시간: %.2fs" % (time.time() - start_time))


0번째 epoch 시작
Training loss (for one batch) at step 0 번째 스텝에서 배치별 훈련데이터 손실: 0.3174
현재까지 학습한 데이터 개수: 64
Training loss (for one batch) at step 200 번째 스텝에서 배치별 훈련데이터 손실: 0.5646
현재까지 학습한 데이터 개수: 12864
Training loss (for one batch) at step 400 번째 스텝에서 배치별 훈련데이터 손실: 0.1751
현재까지 학습한 데이터 개수: 25664
Training loss (for one batch) at step 600 번째 스텝에서 배치별 훈련데이터 손실: 0.4162
현재까지 학습한 데이터 개수: 38464
Training loss (for one batch) at step 800 번째 스텝에서 배치별 훈련데이터 손실: 0.4638
현재까지 학습한 데이터 개수: 51264
epoch별 학습데이터 정확도: 1.4870
검증데이터 정확도 : 1.3589
걸린 시간: 1.78s

1번째 epoch 시작
Training loss (for one batch) at step 0 번째 스텝에서 배치별 훈련데이터 손실: 0.1739
현재까지 학습한 데이터 개수: 64
Training loss (for one batch) at step 200 번째 스텝에서 배치별 훈련데이터 손실: 0.3054
현재까지 학습한 데이터 개수: 12864
Training loss (for one batch) at step 400 번째 스텝에서 배치별 훈련데이터 손실: 0.3826
현재까지 학습한 데이터 개수: 25664
Training loss (for one batch) at step 600 번째 스텝에서 배치별 훈련데이터 손실: 0.3235
현재까지 학습한 데이터 개수: 38464
Training loss (for one batch) at step 800 번째 스텝에서 배치별 훈련데이터 손실: 0.2700
현재까지 학습한